In [1]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers
!pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


In [38]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
from google.colab import files
import io

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


data = files.upload()

KeyboardInterrupt: ignored

In [39]:
df2 = pd.read_csv(io.BytesIO(data['balanced.tsv']), sep='\t')
# only pick ones without nan
df = df2[df2['labels'].notna()]
df = pd.DataFrame({
    'text': df['text'].replace(r'\n', ' ', regex=True),
    'labels': df['labels'].astype(int)
})
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train

,text,labels
565,If you look at the temperatures from lowest to...,1
536,because some are a high temperature and the mo...,1
156,I found that the butterflies grow more in warm...,1
587,when you go to the 1960 the temps were 84 and ...,1
426,maybe it is because of partly true/ partly False,0
...,...,...
468,according to the chart higher temperatures see...,1
689,I think the claim is true because the graph sh...,1
547,the weather makes the monarch slow done if it ...,1
777,In 1960 Temperature was 84.56 and in 2016 the ...,1


In [40]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'output_dir': 'output_dir_balanced_data'}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to output_dir_balanced_data.


(85, 0.5483159791020786)

In [44]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.5232236671256434, 'tp': 61, 'tn': 68, 'fp': 26, 'fn': 15, 'eval_loss': 0.49725673686374317}


In [14]:

# The following training uses EDA dataset
train_data = files.upload()
train = pd.read_csv(io.BytesIO(train_data['eda_train.tsv']), sep='\t')
train = pd.DataFrame({
    'text': train['text'].replace(r'\n', ' ', regex=True),
    'labels': train['labels'].astype(int)
})

train

Saving eda_train.tsv to eda_train (3).tsv


,text,labels
0,i said what i said because of cornix where mor...,1
1,i said what i said because of males where more...,1
2,i said what i said because of inch males where...,1
3,i said what i said because of males where more...,1
4,i made a graph to show whether the temperature...,1
...,...,...
1252,If you look at the chart the year 1960 is lowe...,0
1253,"Yes, because if you look at the temperature it...",0
1254,because for some reason the heat makes the gen...,0
1255,I think claim 2 is false because the populatio...,0


In [15]:
test_data = files.upload()
test = pd.read_csv(io.BytesIO(test_data['test.tsv']), sep='\t')
test = pd.DataFrame({
    'text': test['text'].replace(r'\n', ' ', regex=True),
    'labels': test['labels'].astype(int)
})

test

Saving test.tsv to test (3).tsv


,text,labels
0,I think that the temperature effects how quick...,0
1,I think this claim is true because all the fem...,1
2,I think this claim is true because whether sea...,1
3,Most of the chrysalis temperatures are 60 degr...,1
4,i dont know add add add add and befriends are ...,0
...,...,...
129,I think that this is false because the scienti...,0
130,I think that this claim is false because in th...,0
131,I think its true because number 1 is 2002 its ...,0
132,It is true because when I put the years from l...,0


In [16]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'output_dir': 'eda_output_dir'}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to eda_output_dir.


(158, 0.5906697966252701)

In [19]:
result, model_outputs, wrong_predictions = model.eval_model(train)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6611319790240019, 'tp': 642, 'tn': 401, 'fp': 164, 'fn': 50, 'eval_loss': 0.3628837062470332}


In [20]:
# The following training uses balanced dataset
train_data = files.upload()
train = pd.read_csv(io.BytesIO(train_data['train.tsv']), sep='\t')
train = pd.DataFrame({
    'text': train['text'].replace(r'\n', ' ', regex=True),
    'labels': train['labels'].astype(int)
})

train

Saving train.tsv to train (4).tsv


,text,labels
0,because,0
1,I think this is false because the count goes f...,0
2,The table shows that lower temperatures are ma...,1
3,Yes this is true because I sorted the temps fr...,1
4,it went from 33.50 to 909.50,0
...,...,...
711,as the years go by they get lesser and lesser ...,1
712,the monarch is a caterpillar when it is below ...,1
713,if it is warm out it will change faster than i...,1
714,Some males where born in hot texture same as ...,1


In [21]:
test_data = files.upload()
test = pd.read_csv(io.BytesIO(test_data['test.tsv']), sep='\t')
test = pd.DataFrame({
    'text': test['text'].replace(r'\n', ' ', regex=True),
    'labels': test['labels'].astype(int)
})

test

Saving test.tsv to test (4).tsv


,text,labels
0,"I think this clam is true, Because in the stor...",0
1,Most of the chrysalis temperatures are 60 degr...,1
2,I think it has to do with the turtle populatio...,0
3,I think that this is true because mostly the f...,1
4,"I think its true because in colder weather, mo...",0
...,...,...
129,In 1993 there was about 316 butterflies but in...,0
130,I think that this is false because the scienti...,0
131,I think its false because the numbers go up an...,0
132,86.72 it hier than 85.21 86.72=2020 . 85...,0


In [22]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'output_dir': 'balanced_output_dir'}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train)

result, model_outputs, wrong_predictions = model.eval_model(test)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to balanced_output_dir.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.49506334345690495, 'tp': 31, 'tn': 67, 'fp': 32, 'fn': 4, 'eval_loss': 0.6878364717259127}
